In [33]:
import os

# Check these before running

# Finn's Laptop
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir
codedir = os.path.join(studydir,'code/kpum_noddi')
datadirbasefolder = 'derivatives/dMRI_OrigProtocol'
protocol = 'ORIG' # 'NEW'

# KPUM Workstation
#studydir = '/mnt/e/Research/Projects/KPUM_NODDI/Data'
#niftibasefolder = 'nifti' # nifti basefolder to go within studydir
#codedir = os.path.join('home/radio/KPUM_NODDI','code/kpum_noddi')
#datadirbasefolder = 'derivatives/dMRI_OrigProtocol'
#protocol = 'ORIG' # 'NEW'

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, datadirbasefolder)
subjecttrackerfile = 'Subject_Tracker_for_dmri_pipeline.tsv'

###################################################################################
# USER INPUT

# Participant details
subject = '023'
session = 'MR1'

In [34]:
import os

# Define I/O folders and files
niftifolder = os.path.join(studydir, niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
datadir = os.path.join(studydir, datadirbasefolder,  os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)


In [35]:
######################################
# Define local functions

def create_subjectrackertsv(subjecttrackertsv, subject, session) :   
    import os 
    import pandas as pd
    # create dataframe and write to file
    new_row = {'participant_id': [f'sub-{subject}'], 
            'session_id': [f'ses-{session}']} 
    df = pd.DataFrame(new_row)
    df.to_csv(subjecttrackertsv, sep="\t", index=False)

def add_rowtodataframe(df, subject, session) :
    import pandas as pd
    # Check if {subject} {session} has entry in DataFrame, and adds if not
    if not ((df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}')).any() :
        # Add as a new line at the bottom
        df = pd.concat([df, *[df.tail(1)]*1], ignore_index=True) # duplicates the last row in df
        # first set all columns in the new last row to ''
        df.loc[df.index[-1],:] = ''
        # then update with correct subject and session
        df.loc[df.index[-1],'participant_id'] = f'sub-{subject}'
        df.loc[df.index[-1],'session_id'] = f'ses-{session}'
    return df

def insert_process(df, process) :
    import pandas as pd
    # Check if process is in any of the df.columns
    # if not include put
    if not process in df.columns :
        # insert "process" and "process comments" columns add new two last columnsS
        df.insert(loc=len(df.columns), column=process, value='')
        df.insert(loc=len(df.columns), column=process+' comments', value='')
    return df

def checkstatus_process(df, process, subject, session, status) :
    import pandas as pd
    # Check the status of process for {subject} {session}
    outputboolean = df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), process ].eq(status).any()
    return outputboolean

def setstatus_process(df, process, subject, session, status) :
    import pandas as pd
    # Check the status of process for {subject} {session}
    df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), process ] = status
    return df

def perform_process(processcall) :
    import os, subprocess
    # Perform the process given by the processcall by launching into terminal
    p=subprocess.Popen(processcall, stdout=subprocess.PIPE, shell=True)
    # Poll process.stdout to show stdout live
    while True:
        output = p.stdout.readline()
        if p.poll() is not None:
            break
        if output:
            print(output.strip().decode("utf-8"))
    rc = p.poll()

########################################
## START

import os 
import pandas as pd

currdir = os.getcwd()
os.chdir(studydir)

# Read the subjecttrackertsv-file
if not os.path.isfile(subjecttrackertsv) :
    print(f'Creating {subjecttrackertsv}')
    create_subjectrackertsv(subjecttrackertsv, subject, session)
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Check if subject session has an entry in subjecttrackertsv and add if not add one
df = add_rowtodataframe(df, subject, session)

## Process to perform - dmri_prepare_pipeline
process = 'dmri_prepare_pipeline'
processfile = 'dmri_prepare_pipeline.sh'
processfilepath = os.path.join(codedir,'shell', processfile)
processcall = f"bash {processfilepath} {subject} {session} -d {datadir}" 
print(f'Process to perform {process}')
if not process in df.columns :
    insert_process(df, process)
# IMPORTANT - Should have a check here on whether to perform, but cannot make it work 
# Decide if we are going to perform process (i.e. Status should be Pending) and perform process
if checkstatus_process(df, process, subject, session, 'Stop') :
    print('We have to stop here')
elif checkstatus_process(df, process, subject, session, 'Done') :
    print(f'Process {process} is already Done')
else :
    print(f'Status for {process} is set to Pending = need to do this step')
    setstatus_process(df, process, subject, session, 'Pending')
    perform_process(processcall)
    # Update status if process has gone to its end
    if os.path.exists(os.path.join(datadir,'dwi/orig')) :
        print(f'Status for {process} is set to Done')
        setstatus_process(df, process, subject, session, 'Done')
print()

## Process to perform - dmri_preprocess
process = 'dmri_preprocess'
processfile = 'dmri_preprocess.sh'
processfilepath = os.path.join(codedir,'shell', processfile)
sessionQCfile = os.path.join(datadir,'session_QC.tsv')
threads = 4
processcall = f"bash {processfilepath} {subject} {session} -d {datadir} -protocol {protocol} -s {sessionQCfile} -threads {threads}" 
print(f'Process to perform {process}')
if not process in df.columns :
    insert_process(df, process)
# IMPORTANT - Should have a check here on whether to perform, but cannot make it work 
# Decide if we are going to perform process (i.e. Status should be Pending) and perform process
if checkstatus_process(df, process, subject, session, 'Stop') :
    print('We have to stop here')
elif checkstatus_process(df, process, subject, session, 'Done') :
    print(f'Process {process} is already Done')
else :
    print(f'Status for {process} is set to Pending = need to do this step')
    setstatus_process(df, process, subject, session, 'Pending')
    perform_process(processcall)
    # Update status if process has gone to its end (here last file written in /dwi)
    if os.path.isfile(os.path.join(datadir,'dwi',f'sub-${subject}_ses-${session}_space-dwi_mask.nii')) :
        print(f'Status for {process} is set to Done')
        setstatus_process(df, process, subject, session, 'Done')
print()

## Process to perform - dmri_dtidk
process = 'dmri_dtidki'
processfile = 'dmri_dtidki.sh'
processfilepath = os.path.join(codedir,'shell', processfile)
dwi = os.path.join(datadir, 'dwi', f'sub-{subject}_ses-{session}_desc-preproc-inorm_dwi.mif')
mask = os.path.join(datadir, 'dwi', f'sub-{subject}_ses-{session}_space-dwi_mask.mif')
threads = 4
processcall = f"bash {processfilepath} {subject} {session} -d {datadir} -dwi {dwi} -threads {threads} -mask {mask}" 
print(f'Process to perform {process}')
if not process in df.columns :
    insert_process(df, process)
# IMPORTANT - Should have a check here on whether to perform, but cannot make it work 
# Decide if we are going to perform process (i.e. Status should be Pending) and perform process
if checkstatus_process(df, process, subject, session, 'Stop') :
    print('We have to stop here')
elif checkstatus_process(df, process, subject, session, 'Done') :
    print(f'Process {process} is already Done')
else :
    setstatus_process(df, process, subject, session, 'Pending')
    perform_process(processcall)
    # Update status if process has gone to its end (here last file written in /dwi)
    if os.path.isfile(os.path.join(datadir,'dwi','dki',f'sub-${subject}_ses-${session}_desc-preproc-inorm_rk.nii')) :
        print(f'Status for {process} is set to Done')
        setstatus_process(df, process, subject, session, 'Done')
print()

# End by writing to subjecttrackertsv
df = df.sort_values(by = 'participant_id')
df.to_csv(subjecttrackertsv, sep="\t", index=False)
# and back to currdir
os.chdir(currdir)


Process to perform dmri_prepare_pipeline
Status for dmri_prepare_pipeline is set to Pending = need to do this step
Preparing for dMRI pipeline
Subject:           023
Session:           MR1
Session file:      nifti/sub-023/ses-MR1/session_QC.tsv
Data directory:    /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigProtocol/sub-023/ses-MR1

/mnt/s/Research/Projects/KPUM_NODDI/Data/code/kpum_noddi/shell/dmri_prepare_pipeline.sh       023 MR1 -d /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigProtocol/sub-023/ses-MR1
----------------------------
dMRI preprocessing on subject 023 and session MR1

Transfer data in nifti/sub-023/ses-MR1/session_QC.tsv which has qc_pass_fail = 1 or 0.5
Status for dmri_prepare_pipeline is set to Done

Process to perform dmri_preprocess
Status for dmri_preprocess is set to Pending = need to do this step
dMRI preprocessing
Subject:       	023
Session:        MR1
Session file:	/mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigP

mrconvert: [100%] copying from "/mnt/s/Res...-023_ses-MR1_dir-AP_dwi.nii" to "/mnt/s/Res...1/dwi/preproc/tmp_dwiAP.mif"[0K
dwiextract: [100%] extracting volumes[0K
dwiextract: [100%] extracting volumes[0K
mrconvert: [100%] copying from "/tmp/mrtrix-tmp-GaaskD.mif" to "/mnt/s/Res...wi/preproc/tmp_dwiAP_b0.mif"[0K
mrcat: [100%] concatenating "/mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigProtocol/sub-023/ses-MR1/dwi/preproc/tmp_dwiAP_b0.mif"[0K
mrcat: [100%] concatenating "/mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigProtocol/sub-023/ses-MR1/dwi/preproc/tmp_dwiAP_b1000b2000.mif"[0K
mrconvert: [100%] copying from "/mnt/s/Res...-023_ses-MR1_dir-AP_dwi.nii" to "/tmp/mrtrix-tmp-rWt14e.mif"[0K
mrconvert: [100%] copying from "/tmp/mrtrix-tmp-rWt14e.mif" to "/mnt/s/Res.../dwi/preproc/topup/b0AP.mif"[0K


We only have b0AP.mif in /topup => do not have a fieldmap, so eddy will be run without a TOPUP fieldmap


mrconvert: [100%] copying from "dwiAP.mif" to "dwi.mif"[0K


Doing MP PCA-denosing with dwidenoise


dwidenoise: [100%] preloading data for "dwi.mif"[0K
dwidenoise: [ 24%] running MP-PCA denoising...